# RAG Testing

In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install ipywidgets
!pip install langchain-huggingface


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
# Specify the dataset name and the column containing the content
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"  # or any other column you're interested in

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

# Display the first 15 entries
data[:2]

[Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}, page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."'),
 Document(metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'response': 'Tope', 'category': 'classification'}, page_content='""')]

In [4]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [5]:
docs[0]

Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}, page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."')

# load the model for embedding the texts

In [6]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [7]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.03833850845694542, 0.1234646737575531, -0.028642946854233742]

# storing vectors into a FAISS Vector database
-it embeds each of the chunks during the storing

In [8]:
db = FAISS.from_documents(docs, embeddings)

In [9]:
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

"The goal of cheese making is to control the spoiling of milk into cheese. The milk is traditionally from a cow, goat, sheep or buffalo, although, in theory, cheese could be made from the milk of any mammal. Cow's milk is most commonly used worldwide. The cheesemaker's goal is a consistent product with specific characteristics (appearance, aroma, taste, texture). The process used to make a Camembert will be similar to, but not quite the same as, that used to make Cheddar.\n\nSome cheeses may be deliberately left to ferment from naturally airborne spores and bacteria; this approach generally leads to a less consistent product but one that is valuable in a niche market.\n\nCulturing\nCheese is made by bringing milk (possibly pasteurised) in the cheese vat to a temperature required to promote the growth of the bacteria that feed on lactose and thus ferment the lactose into lactic acid. These bacteria in the milk may be wild, as is the case with unpasteurised milk, added from a culture,


# load the fine_tuned model for tokenize the query

In [10]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

Invalid model-index. Not loading eval results into CardData.


In [11]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

Device set to use cpu
Invalid model-index. Not loading eval results into CardData.


In [12]:
from transformers import pipeline
generator = pipeline('text-generation', model = 'HuggingFaceH4/zephyr-7b-beta')
generator("Hello, I'm a language model", max_length = 30, num_return_sequences=3)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: Greedy methods without beam search do not support `num_return_sequences` different than 1 (got 3).

In [13]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever()

In [14]:
docs = retriever.invoke("What is Cheesemaking?")
print(docs[0].page_content)

"The goal of cheese making is to control the spoiling of milk into cheese. The milk is traditionally from a cow, goat, sheep or buffalo, although, in theory, cheese could be made from the milk of any mammal. Cow's milk is most commonly used worldwide. The cheesemaker's goal is a consistent product with specific characteristics (appearance, aroma, taste, texture). The process used to make a Camembert will be similar to, but not quite the same as, that used to make Cheddar.\n\nSome cheeses may be deliberately left to ferment from naturally airborne spores and bacteria; this approach generally leads to a less consistent product but one that is valuable in a niche market.\n\nCulturing\nCheese is made by bringing milk (possibly pasteurised) in the cheese vat to a temperature required to promote the growth of the bacteria that feed on lactose and thus ferment the lactose into lactic acid. These bacteria in the milk may be wild, as is the case with unpasteurised milk, added from a culture,


In [15]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [17]:
def err_remove(er):
 lin = "------------"
 er = str(er)
 start_index = er.find(lin) + len(lin)
 end_index = er.rfind(lin)
 ans = er[start_index:end_index].strip()
 return ans

In [19]:
question = "Who is Hamlet ?"
result = qa.invoke({"query": question})
result["result"]



C:\Users\Jeff Lam\AppData\Roaming\Python\Python312\site-packages\transformers\pipelines\question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


ValueError: Context information is below. 
------------
"Shakespeare produced most of his known works between 1589 and 1613. His early plays were primarily comedies and histories and are regarded as some of the best works produced in these genres. He then wrote mainly tragedies until 1608, among them Hamlet, Romeo and Juliet, Othello, King Lear, and Macbeth, all considered to be among the finest works in the English language. In the last phase of his life, he wrote tragicomedies (also known as romances) and collaborated with other playwrights."
------------
Given the context information and not prior knowledge, answer the question: Who is Hamlet ?
 argument needs to be of type (SquadExample, dict)

In [1]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# 1. Set up a text2text-generation pipeline.
#    We use device=-1 to ensure it runs on the CPU.
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    max_length=512,
    device=-1
)

# 2. Wrap it in the LangChain pipeline object
llm = HuggingFacePipeline(pipeline=pipe)

# 3. Re-create the RetrievalQA chain with the new, correct LLM.
#    The 'retriever' object is already correctly defined from the previous cells.
#    We'll use the 'stuff' chain type, which is standard and efficient. It "stuffs" all
#    retrieved documents into the prompt for the LLM.
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False
)

# 4. Ask the original question again. This will now work correctly.
question = "Who is Hamlet?"
result = qa.invoke({"query": question})
print(f"Question: {question}")
print(f"Answer: {result['result']}")
print("-" * 30)


# 5. Let's try another question to confirm the RAG pipeline is working.
question_2 = "When did Virgin Australia start operating?"
result_2 = qa.invoke({"query": question_2})
print(f"Question: {question_2}")
print(f"Answer: {result_2['result']}")

config.json: 0.00B [00:00, ?B/s]

C:\Users\Jeff Lam\PycharmProjects\PythonProject\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jeff Lam\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/google/flan-t5-base/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use -1


NameError: name 'RetrievalQA' is not defined

In [2]:
# ==============================================================================
# 1. INSTALLATION
# ==============================================================================
# Install all the required libraries. The '-q' flag is for a quiet installation.
!pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu ipywidgets langchain-huggingface


In [3]:
# ==============================================================================
# 2. IMPORTS
# ==============================================================================
# Core LangChain and data loading components
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Hugging Face specific components
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
# ==============================================================================
# 3. DATA LOADING AND PREPARATION
# ==============================================================================
print("Step 3: Loading and preparing data...")

# Load the dataset from Hugging Face Hub
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()

# Split the loaded documents into smaller, manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)

print("Data loaded and split into chunks.")

Step 3: Loading and preparing data...
Data loaded and split into chunks.


In [5]:
# ==============================================================================
# 4. EMBEDDING MODEL AND VECTOR STORE
# ==============================================================================
print("\nStep 4: Setting up embedding model and vector store...")

# Define the embedding model to convert text chunks into numerical vectors
model_path = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Create the FAISS vector store to efficiently search the document embeddings
# This process may take a few minutes as it embeds all document chunks
db = FAISS.from_documents(docs, embeddings)

print("FAISS vector store created successfully.")


Step 4: Setting up embedding model and vector store...
FAISS vector store created successfully.


In [6]:
# ==============================================================================
# 5. RETRIEVER, LLM, AND RAG CHAIN SETUP
# ==============================================================================
print("\nStep 5: Setting up the Retriever, LLM, and RAG chain...")

# Create a retriever from the vector store to fetch relevant documents
retriever = db.as_retriever(search_kwargs={"k": 4})

# Set up the Language Model (LLM) pipeline
# We use a 'text2text-generation' model which is suitable for question-answering based on context
# The 'device=-1' argument ensures the model runs on the CPU
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    max_length=512,
    device=-1
)

llm = HuggingFacePipeline(pipeline=pipe)

# Create the final Retrieval-Augmented Generation (RAG) chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 'stuff' chain type passes all retrieved chunks to the LLM
    retriever=retriever,
    return_source_documents=False
)

print("RAG chain is ready.")


Step 5: Setting up the Retriever, LLM, and RAG chain...


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


RAG chain is ready.


In [7]:
# ==============================================================================
# 6. EXECUTE QUERIES
# ==============================================================================
print("\nStep 6: Executing queries...")

# --- Query 1 ---
question_1 = "Who is Hamlet?"
result_1 = qa_chain.invoke({"query": question_1})
print(f"\nQuestion: {question_1}")
print(f"Answer: {result_1['result']}")
print("-" * 50)

# --- Query 2 ---
question_2 = "When did Virgin Australia start operating?"
result_2 = qa_chain.invoke({"query": question_2})
print(f"Question: {question_2}")
print(f"Answer: {result_2['result']}")
print("-" * 50)

# --- Query 3 ---
question_3 = "What is cheesemaking?"
result_3 = qa_chain.invoke({"query": question_3})
print(f"Question: {question_3}")
print(f"Answer: {result_3['result']}")
print("-" * 50)


Step 6: Executing queries...


Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors



Question: Who is Hamlet?
Answer: Shakespeare
--------------------------------------------------
Question: When did Virgin Australia start operating?
Answer: 31 August 2000
--------------------------------------------------
Question: What is cheesemaking?
Answer: control the spoiling of milk into cheese
--------------------------------------------------
